In [39]:
import pandas as pd
import geopandas as gpd
import numpy as np
import pygeohash
import folium

# Exploration & Analysis
Now we analyze the results to see look for correlations

In [6]:
crime_df = pd.read_json('./prepared_data/crash_data.json', orient='records')
intersection_df = pd.read_json('./prepared_data/intersection_info.json', orient='records') 

In [13]:
data_df = pd.merge(crime_df, intersection_df, how='left', on=['geohash'])
data_df.head()

,objectid,case_no,date,description,location,city,latitude,longitude,globalid,geohash,street1,street2,street_types,owners,max_width,max_speed_limit,min_speed_limit,max_lanes,min_lanes
0,3,202200349491,2022-11-25 21:33:48+00:00,MVC - PI,GUELPH LI,BURLINGTON,43.350226,-79.801527,f623ed61-fbc7-4391-8038-9e385d6ef3dd,hcyf42wd,HARVESTER RD,GUELPH LINE,"[MINOR, MAJOR]","[COB, ROH]",9999.0,60.0,50.0,8.0,5.0
1,10,202200351471,2022-11-27 23:46:00+00:00,MVC - PI,GUELPH LI,BURLINGTON,43.338309,-79.785323,aff64172-9cc4-44b0-a78b-71d0c21177c8,hcyf45tb,NEW ST,GUELPH LINE,[MAJOR],[COB],0.0,60.0,50.0,6.0,5.0
2,13,202200353214,2022-11-29 15:40:00+00:00,MVC - HIT & RUN,GUELPH LI,BURLINGTON,43.364656,-79.822041,8883b2d6-9c65-4f61-a25d-5e9e3d77d4d9,hcycfuc4,UPPER MIDDLE RD,GUELPH LINE,[MAJOR],"[COB, ROH]",9999.0,60.0,60.0,6.0,5.0
3,21,202200357807,2022-12-04 17:44:48+00:00,MVC - PI,GUELPH LI,BURLINGTON,43.347329,-79.797437,48573cd9-b167-4c66-8bc7-7d6e32f546ee,hcyf43kd,FAIRVIEW ST,GUELPH LINE,[MAJOR],"[COB, ROH]",9999.0,60.0,50.0,9.0,7.0
4,24,202200361004,2022-12-08 00:04:00+00:00,MVC - PI,GUELPH LI,BURLINGTON,43.352178,-79.803863,6c97ac07-1a63-41b5-95f8-725dea1811f4,hcyf480h,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
data_df[data_df['street1'].isna()].count()

,objectid,case_no,date,description,location,city,latitude,longitude,globalid,geohash,street1,street2,street_types,owners,max_width,max_speed_limit,min_speed_limit,max_lanes,min_lanes
4,24,202200361004,2022-12-08 00:04:00+00:00,MVC - PI,GUELPH LI,BURLINGTON,43.352178,-79.803863,6c97ac07-1a63-41b5-95f8-725dea1811f4,hcyf480h,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,29,202200366735,2022-12-08 16:15:00+00:00,MVC - HIT & RUN,GUELPH LI,BURLINGTON,43.352178,-79.803863,2cc4e005-eb9c-40d6-af1a-31488d1f6f9a,hcyf480h,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,31,202200367839,2022-12-14 22:20:00+00:00,MVC - PI,GUELPH LI,BURLINGTON,43.352178,-79.803863,25e4a0bc-6bb9-4f4d-a64b-3b79b1b51638,hcyf480h,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,32,202200367890,2022-12-14 23:25:00+00:00,MVC - HIT & RUN,GUELPH LI,BURLINGTON,43.352178,-79.803863,58a8b2b4-2072-49d2-b348-f4e483df65c7,hcyf480h,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,46,202200376740,2022-12-24 19:37:00+00:00,MVC - PI,GUELPH LI,BURLINGTON,43.432284,-79.905342,10e5a608-bc2b-44d7-a986-ce96f39157b0,hcyckmdm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,13831,202300276459,2023-08-29 17:19:00+00:00,MVC - HIT & RUN,403 HWY,BURLINGTON,43.312032,-79.859808,e22c4a8f-1bc2-42a7-9360-cfd92dd45e57,hcyc9tfy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1096,13833,202300293403,2023-09-12 16:50:00+00:00,MVC - PI,403 HWY,BURLINGTON,43.312032,-79.859808,5c332532-35b8-4dd8-b124-e702eb27111f,hcyc9tfy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1097,13835,202300350682,2023-11-07 17:25:00+00:00,MVC - HIT & RUN,403 HWY,BURLINGTON,43.312032,-79.859808,05fd93a3-3c17-44cd-be11-7b4c4abaca4b,hcyc9tfy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1098,13899,202300191708,2023-06-17 03:17:00+00:00,MVC - HIT & RUN,MARTHA ST,BURLINGTON,43.327330,-79.793532,2b973367-ce54-4d2b-94b3-b0e3e7ee2d18,hcyf1fjb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Looks like some points aren't properly matched 

In [14]:
data_df['intersection_name'] = data_df.apply(lambda x: x['street1'] + x['street2'], axis=1)

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [ ]:
tdf = data_df.copy()
tdf['intersection_name'] = 

In [37]:
intersection_df[intersection_df['street2'] == 'MARTHA ST']

,geohash,street1,street2,street_types,owners,max_width,max_speed_limit,min_speed_limit,max_lanes,min_lanes
225,hcyf1crc,CAROLINE ST,MARTHA ST,[MINOR],[COB],20.0,40.0,40.0,2.0,2.0
228,hcyf1crr,MARIA ST,MARTHA ST,[MINOR],[COB],20.0,40.0,40.0,3.0,2.0
245,hcyf1fn0,PINE ST,MARTHA ST,[MINOR],[COB],20.0,50.0,50.0,2.0,2.0
253,hcyf1fjr,LAKESHORE RD,MARTHA ST,"[MINOR, MAJOR]",[COB],20.0,50.0,50.0,5.0,2.0
1762,hcyf1cwv,NEW ST,MARTHA ST,"[MINOR, MAJOR]",[COB],20.0,50.0,40.0,4.0,2.0


In [62]:
position =  (43.31510324241185, -79.80731370830324)
m = folium.Map(location=position, zoom_start=20)
folium.Marker(location=position).add_to(m)
opos = (43.315486907958984, -79.80683326721191)
folium.Marker(location=opos, popup='other').add_to(m)
del position
m